In [2]:
% matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

In [3]:
H1_data = np.genfromtxt('H1_times.csv')

ValueError: Some errors were detected !
    Line #353 (got 1 columns instead of 12)
    Line #354 (got 1 columns instead of 12)
    Line #355 (got 1 columns instead of 12)
    Line #356 (got 1 columns instead of 12)
    Line #357 (got 1 columns instead of 12)
    Line #358 (got 1 columns instead of 12)
    Line #359 (got 1 columns instead of 12)
    Line #360 (got 1 columns instead of 12)
    Line #361 (got 1 columns instead of 12)
    Line #362 (got 1 columns instead of 12)
    Line #363 (got 1 columns instead of 12)
    Line #364 (got 1 columns instead of 12)
    Line #515 (got 1 columns instead of 12)
    Line #516 (got 1 columns instead of 12)
    Line #517 (got 1 columns instead of 12)
    Line #518 (got 1 columns instead of 12)
    Line #519 (got 1 columns instead of 12)
    Line #520 (got 1 columns instead of 12)
    Line #521 (got 1 columns instead of 12)
    Line #522 (got 1 columns instead of 12)
    Line #523 (got 1 columns instead of 12)
    Line #524 (got 1 columns instead